1) Rappresentare query e profili come embedding 

In [2]:
from sentence_transformers import SentenceTransformer

# Carica il modello pre-addestrato
model = SentenceTransformer('all-MiniLM-L6-v2')

# Funzione per calcolare embedding di una query
def get_embedding(query):
    return model.encode(query)

# Prova con una query di esempio
query = "eventi scientifici a Verona"
embedding = get_embedding(query)

print("Embedding calcolato per la query:")
print(embedding)

Embedding calcolato per la query:
[-9.12196338e-02  4.82078306e-02  4.28559585e-03  5.62877618e-02
 -9.31479782e-02  3.17956097e-02  5.28045185e-02  4.10347991e-02
  9.86863207e-03 -1.68792587e-02  6.44217581e-02 -1.30243286e-01
 -1.04704853e-02  5.50178215e-02 -8.24869350e-02 -3.65489200e-02
  8.48140568e-03  1.25394249e-02 -3.26627120e-02  1.95095856e-02
 -2.53478643e-02 -6.07474260e-02  2.42216373e-03  3.99483554e-02
 -5.62509447e-02 -1.42766081e-03 -1.71665568e-02 -3.82813960e-02
 -1.19715547e-02  2.38501560e-02  1.69824734e-02  9.05974060e-02
  3.18437144e-02 -9.95773077e-02  6.82917461e-02 -2.22271681e-02
 -2.03441475e-02 -5.04771732e-02  1.00376204e-01  7.06631318e-02
 -4.29129601e-02 -4.38364595e-02  4.70876694e-02  3.04056853e-02
  5.08238375e-02 -5.68662025e-02  1.71408765e-02  3.65497656e-02
 -1.84695143e-02  2.36278102e-02 -1.40578479e-01  1.90707133e-03
 -6.96761087e-02 -5.59170581e-02 -3.46701033e-02  3.95653881e-02
  4.56602834e-02 -5.17620109e-02  1.80368237e-02 -4.8605

2) Costruire un archivio di entità semplificato basato su keyword o profili ORCID

In [3]:
from collections import Counter

def costruisci_profilo_entita(lista_query):
    # Semplice estrazione di keyword da query (qui split su spazi, potresti migliorare con NLP)
    entita = []
    for query in lista_query:
        parole = query.lower().split()
        entita.extend(parole)
    # Conta frequenze
    profilo = Counter(entita)
    return profilo


3) Calcolare similarità tra query e entità del profilo usando embedding

In [4]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

model = SentenceTransformer('all-MiniLM-L6-v2')

def embedding_testo(testo):
    return model.encode(testo)

def similarita(query, entita_list):
    emb_query = embedding_testo(query).reshape(1, -1)
    emb_entita = [embedding_testo(e).reshape(1, -1) for e in entita_list]
    emb_entita = np.vstack(emb_entita)
    sim = cosine_similarity(emb_query, emb_entita)[0]
    return dict(zip(entita_list, sim))


4) Suggerire la next query semplificata

In [7]:
def suggerisci_next_query(query_corrente, profilo_entita, top_k=3):
    entita_list = list(profilo_entita.keys())
    sim_scores = similarita(query_corrente, entita_list)
    # Ordina per similarità decrescente
    top_entita = sorted(sim_scores.items(), key=lambda x: x[1], reverse=True)
    
    # Parole già presenti nella query corrente
    parole_query = set(query_corrente.lower().split())
    
    # Seleziona entità diverse da quelle già presenti
    entita_filtrate = [e for e in top_entita if e[0] not in parole_query]
    
    # Prendi le prime top_k entità senza ripetizioni
    top_entita_finali = entita_filtrate[:top_k]
    
    suggerita = query_corrente + " " + " ".join([e[0] for e in top_entita_finali])
    return suggerita



5) Esempio d’uso completo:

In [8]:
# Lista query esempio per costruire il profilo
query_passate = [
    "eventi scientifici a Verona",
    "conferenze di intelligenza artificiale",
    "meteorologia e cambiamenti climatici",
    "convegni su ambiente e energia"
]

profilo = costruisci_profilo_entita(query_passate)

query_corrente = "eventi a Verona"
next_query = suggerisci_next_query(query_corrente, profilo)

print("Next query suggerita:", next_query)


Next query suggerita: eventi a Verona cambiamenti climatici scientifici


## Integrazione profilo ORCID tramite API

Qui recuperiamo i dati del profilo ORCID e ne estraiamo le keyword da usare per personalizzare la raccomandazione.


In [9]:
import requests

def get_orcid_profile(orcid_id):
    url = f"https://pub.orcid.org/v3.0/{orcid_id}/person"
    headers = {"Accept": "application/json"}
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Errore nel recupero profilo ORCID: {response.status_code}")
        return None

def estrai_keyword_orcid(profile_json):
    keywords = []
    if 'keywords' in profile_json:
        kw_list = profile_json['keywords'].get('keyword', [])
        for kw in kw_list:
            if 'content' in kw:
                keywords.append(kw['content'].lower())
    return keywords

# Esempio con un ORCID reale
orcid_id = "0000-0001-6092-6831"
profile_data = get_orcid_profile(orcid_id)

if profile_data:
    keywords_orcid = estrai_keyword_orcid(profile_data)
    print("Keyword estratte da ORCID:")
    print(keywords_orcid)
else:
    keywords_orcid = []


Keyword estratte da ORCID:
['database', 'data science', 'ethics in data management', 'recommender systems', 'context-awareness', 'personalization']


estrazione semplificata di keyword + lista query passate

In [10]:
def costruisci_profilo_entita_completo(lista_query, keyword_orcid):
    entita = []
    # Estraggo parole dalle query
    for query in lista_query:
        parole = query.lower().split()
        entita.extend(parole)
    # Aggiungo le keyword ORCID (già in minuscolo)
    entita.extend(keyword_orcid)
    profilo = Counter(entita)
    return profilo


In [11]:
# Supponiamo tu abbia già le query passate e le keyword ORCID
profilo_completo = costruisci_profilo_entita_completo(query_passate, keywords_orcid)

query_corrente = "eventi a Verona"
next_query = suggerisci_next_query(query_corrente, profilo_completo)

print("Next query suggerita (con ORCID):", next_query)


Next query suggerita (con ORCID): eventi a Verona cambiamenti climatici scientifici


Connessione con DATABASE

In [14]:
import psycopg2

def connetti_db():
    conn = psycopg2.connect(
        host="localhost",
        database="tesi_meteo",
        user="postgres",
        password="1234"
    )
    return conn

def cerca_eventi(conn, keyword):
    cur = conn.cursor()
    # Query semplice che cerca keyword nella descrizione degli eventi
    sql = "SELECT id, nome_evento, descrizione, data_evento FROM eventi WHERE descrizione ILIKE %s LIMIT 10"
    cur.execute(sql, ('%' + keyword + '%',))
    risultati = cur.fetchall()
    cur.close()
    return risultati

# Connessione e prova ricerca
conn = connetti_db()
keyword = "scientifici"  # esempio da next query
eventi = cerca_eventi(conn, keyword)

for evento in eventi:
    print(evento)

conn.close()


OperationalError: connection to server at "localhost" (::1), port 5432 failed: FATALE:  il database "tesi_meteo" non esiste
